## **Installing Packages**

In [ ]:
!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## **Importing Packages**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn 
import re
import nltk 
from nltk.corpus import stopwords
from textblob import Word
import spacy
  

from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# **Reading Data From Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
Data_Essay_01 = pd.read_csv("/content/drive/MyDrive/IntelliTech-DataSet/EssaySet01.csv")
Data_Essay_01.head()

Mounted at /content/drive


,ID,Essay,Rater_1 Score,Rater_2 Score,Total Score
0,1,"Dear local newspaper, I think effects computer...",4.0,4.0,8.0
1,2,"Dear @CAPS1 @CAPS2, I believe that using compu...",5.0,4.0,9.0
2,3,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4.0,3.0,7.0
3,4,"Dear Local Newspaper, @CAPS1 I have found that...",5.0,5.0,10.0
4,5,"Dear @LOCATION1, I know having computers has a...",4.0,4.0,8.0


# **Feature Extraction**

## **Essay Pre Processing**

In [ ]:
def Remove_NER(Essay):
  """
    Removes Named Entity Recognition (NER) from each essay

    Args:
      Sentence: Essay of each student 
    
    Returns: 
      String

  """
  return ' '.join (word for word in Essay.lower().split(' ') if not word.startswith('@'))

def Remove_Punctuations(sentence):
  """
    Removes punctuations from text
    Args:
      sentence: Essay of each student
    
    Returns: 
      String
  """
  punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  newSentence = ""
  for word in sentence:
      if (word in punctuations):
          newSentence = newSentence + " "
      else: 
          newSentence = newSentence + word
  return newSentence

def LowerCase_Words(Essay):
  """
    Lower case all the words in an essay

    Args:
      Sentence: Essay of each student
    
    Returns: 
      String
  """
  return re.sub('[0-9]+','', Essay).lower() 

def Tokenize_Essay(Essay):
    Preprocessed = Remove_Punctuations(Essay)
    return " ".join(word_tokenize(Preprocessed))

Removing NERs, Punctuations and Lower Casing

In [ ]:
Data_Essay_01['Preprocessed_Essay'] = Data_Essay_01['Essay'].apply(Remove_NER)
Data_Essay_01['Preprocessed_Essay'] = Data_Essay_01['Preprocessed_Essay'].apply(Tokenize_Essay)
Data_Essay_01.head()

,ID,Essay,Rater_1 Score,Rater_2 Score,Total Score,Preprocessed_Essay
0,1,"Dear local newspaper, I think effects computer...",4.0,4.0,8.0,dear local newspaper i think effects computers...
1,2,"Dear @CAPS1 @CAPS2, I believe that using compu...",5.0,4.0,9.0,dear i believe that using computers will benef...
2,3,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4.0,3.0,7.0,dear more and more people use computers but no...
3,4,"Dear Local Newspaper, @CAPS1 I have found that...",5.0,5.0,10.0,dear local newspaper i have found that many ex...
4,5,"Dear @LOCATION1, I know having computers has a...",4.0,4.0,8.0,dear i know having computers has a positive ef...


## **Basic Count Features**

#### 1. Counting Sentences per Essay

In [ ]:
def Sentence_Count(Essay):
    """
    Counts sentences in an essay

    Args:
      Essay: Essay of each student 
    
    Returns: 
      int
      
  """
    sentence_no = nltk.sent_tokenize(Essay)
    return len(sentence_no)
  
Data_Essay_01['Sent_Count'] = Data_Essay_01['Essay'].apply(Sentence_Count)
Data_Essay_01

,ID,Essay,Rater_1 Score,Rater_2 Score,Total Score,Preprocessed_Essay,Sent_Count
0,1,"Dear local newspaper, I think effects computer...",4.0,4.0,8.0,dear local newspaper i think effects computers...,16
1,2,"Dear @CAPS1 @CAPS2, I believe that using compu...",5.0,4.0,9.0,dear i believe that using computers will benef...,20
2,3,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4.0,3.0,7.0,dear more and more people use computers but no...,14
3,4,"Dear Local Newspaper, @CAPS1 I have found that...",5.0,5.0,10.0,dear local newspaper i have found that many ex...,27
4,5,"Dear @LOCATION1, I know having computers has a...",4.0,4.0,8.0,dear i know having computers has a positive ef...,30
...,...,...,...,...,...,...,...
1778,1783,"Dear @CAPS1, @CAPS2 several reasons on way I t...",4.0,4.0,8.0,dear several reasons on way i that advances in...,21
1779,1784,Do a adults and kids spend to much time on the...,3.0,4.0,7.0,do a adults and kids spend to much time on the...,18
1780,1785,My opinion is that people should have computer...,4.0,4.0,8.0,my opinion is that people should have computer...,18
1781,1786,"Dear readers, I think that its good and bad to...",1.0,1.0,2.0,dear readers i think that its good and bad to ...,1


#### 2. Counting Words per Essay

**Observation:** These word count are more than the original count coz of nltk tokenization. Punctations are treated as seperate words.


In [ ]:
def Word_Count(Essay):
  """
    Counts words in an essay

    Args:
      Essay: Essay of each student 
    
    Returns: 
      int
      
  """
  #cleaned_essay = re.sub('[^a-zA-Z]','',essay) 
  word_no = nltk.word_tokenize(Essay)
  return len(word_no)
 
Data_Essay_01['Word_Count'] = Data_Essay_01['Essay'].apply(Word_Count)
Data_Essay_01.sample()

,ID,Essay,Rater_1 Score,Rater_2 Score,Total Score,Preprocessed_Essay,Sent_Count,Word_Count
351,354,"Dear local Newspaper, @CAPS1 the years compute...",4.0,4.0,8.0,dear local newspaper the years computers has b...,20,302


#### 3. Counting Characters per Essay

In [ ]:
def Char_Count(Essay):
  """
    Counts characters in an essay

    Args:
      Essay: Essay of each student 
    
    Returns: 
      int
      
  """
  #cleaned_essay = re.sub('[^a-zA-Z]',' ',Essay) 
  return len([character for character in Essay])

Data_Essay_01['Char_Count'] = Data_Essay_01['Essay'].apply(Char_Count)
Data_Essay_01.sample()

,ID,Essay,Rater_1 Score,Rater_2 Score,Total Score,Preprocessed_Essay,Sent_Count,Word_Count,Char_Count
1094,1097,"Dear fellow citizens, @CAPS1 think if you had ...",5.0,5.0,10.0,dear fellow citizens think if you had to get a...,23,447,2099


#### 4. Average Word Length of Essay

In [ ]:
def Avg_Word_Count(Essay):
  """
    Calculates Average Word Count In An Essay Set

    Args:
      Essay: Essay of each student 
    
    Returns: 
      float
      
  """
  word_list = nltk.word_tokenize(Essay)
  total = sum(map(len, word_list))/len(word_list)
  return total

Data_Essay_01['Avg_Word_Count'] = Data_Essay_01['Essay'].apply(Avg_Word_Count)
Data_Essay_01.sample()

,ID,Essay,Rater_1 Score,Rater_2 Score,Total Score,Preprocessed_Essay,Sent_Count,Word_Count,Char_Count,Avg_Word_Count
1114,1117,Statistics show that more and more people ever...,3.0,4.0,7.0,statistics show that more and more people ever...,14,372,1793,3.948925


## **Parts Of Speech Counts**

In [ ]:
def Pos_Tag_Count(Essay):
  """
    Counts Parts of Speech in an Essay

    Args:
      Essay: Essay of each student 
    
    Returns: 
      int,int,int,int,int,int
      
  """
  tagged_doc = nlp(Essay)

  adj_count=0
  verb_count=0
  noun_count=0
  pNoun_count=0
  adverb_count=0
  conj_count=0

  for token in tagged_doc:

    if(token.pos_ == 'ADJ'):
      adj_count+=1
    
    elif(token.pos_ =='NOUN'):
      noun_count+=1

    elif (token.pos_ =='PRON'):
      pNoun_count+=1

    elif (token.pos_ =='VERB'):
      verb_count+=1

    elif (token.pos_ =='ADV'):
      adverb_count+=1
    
    elif(token.pos_=='CCONJ'):
      conj_count+=1

  return verb_count,noun_count, adj_count, conj_count, adverb_count,pNoun_count

In [ ]:
Data_Essay_01['Verb_Count'], Data_Essay_01['Noun_Count'], Data_Essay_01['Adj_Count'], Data_Essay_01['Conj_Count'], Data_Essay_01['Adverb_Count'], Data_Essay_01['pNoun_Count']=zip(*Data_Essay_01["Preprocessed_Essay"].map(Pos_Tag_Count))
Data_Essay_01.sample()

,ID,Essay,Rater_1 Score,Rater_2 Score,Total Score,Preprocessed_Essay,Sent_Count,Word_Count,Char_Count,Avg_Word_Count,Verb_Count,Noun_Count,Adj_Count,Conj_Count,Adverb_Count,pNoun_Count
1065,1068,"Computers are a huge impact on society, but do...",5.0,4.0,9.0,computers are a huge impact on society but do ...,32,566,2670,3.791519,83,90,38,25,34,86


## **Mechanics**

#### Counting Spelling Mistakes

In [ ]:
def Check_Spelling(Sentence):
  """
    Checks spelling of each word

    Args:
      word: Words (Tokens) of each essay 
    
    Returns: 
      int
      
  """
  count = 0
  Sentence = word_tokenize(Sentence)
  for word in Sentence:
    word = Word(word)
  
    result = word.spellcheck()

    # result [0][0] contains the bool value if the spelling is correct or not
    # result [0][1] contains the confidence for the suggest correct spelling

    if word != result[0][0]:
      # print(f'Spelling of "{word}" is not correct!')
      # print(f'Correct spelling of "{word}": "{result[0][0]}" (with {result[0][1]} confidence).')
      count = count + 1

  return count

In [ ]:
Data_Essay_01["Preprocessed_Essay"] = Data_Essay_01["Essay"].apply(Remove_NER)
listOfEssays = Data_Essay_01["Preprocessed_Essay"].apply(Remove_Punctuations)

In [ ]:
Data_Essay_01["Spelling_Mistakes_Count"] = zip(*listOfEssays.map(Check_Spelling))

#### Checking Punctuation Mistakes 

In [ ]:
!pip install sentencepiece  
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Restart Runtime after installing this

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification , pipeline

tokenizer = AutoTokenizer.from_pretrained('oliverguhr/fullstop-punctuation-multilang-large')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
text = Data_Essay_01['Preprocessed_Essay'][0]
text

"dear local newspaper, i think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! thing about! dont you think so? how would you feel if your teenager is always on the phone with friends! do you ever time to chat with your friends or buisness partner about things. well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: facebook, myspace ect. just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. how did you learn about other countrys/states outside of yours? well i have by computer/internet, it's a new way to learn about what going on in our time! you might think your child spends a lot of time on the computer, but ask them so question about the economy, sea floor spreading or even about the

In [ ]:
model = AutoModelForTokenClassification.from_pretrained('oliverguhr/fullstop-punctuation-multilang-large')

In [ ]:
pun = pipeline('ner' , model = model , tokenizer = tokenizer)

In [ ]:
# text = Data_Essay_01["Essay"].iloc[1]
output = pun(text)
# print(text)
output

[{'entity': '0',
  'score': 0.9999578,
  'index': 1,
  'word': '▁de',
  'start': 0,
  'end': 2},
 {'entity': '0',
  'score': 0.9997856,
  'index': 2,
  'word': 'ar',
  'start': 2,
  'end': 4},
 {'entity': '0',
  'score': 0.9999435,
  'index': 3,
  'word': '▁local',
  'start': 4,
  'end': 10},
 {'entity': '0',
  'score': 0.5707773,
  'index': 4,
  'word': '▁newspaper',
  'start': 10,
  'end': 20},
 {'entity': '0',
  'score': 0.66460884,
  'index': 5,
  'word': ',',
  'start': 20,
  'end': 21},
 {'entity': '0',
  'score': 0.9999871,
  'index': 6,
  'word': '▁i',
  'start': 21,
  'end': 23},
 {'entity': '0',
  'score': 0.9956696,
  'index': 7,
  'word': '▁think',
  'start': 23,
  'end': 29},
 {'entity': '0',
  'score': 0.99982303,
  'index': 8,
  'word': '▁effects',
  'start': 29,
  'end': 37},
 {'entity': '0',
  'score': 0.999985,
  'index': 9,
  'word': '▁computer',
  'start': 37,
  'end': 46},
 {'entity': '0',
  'score': 0.9999894,
  'index': 10,
  'word': 's',
  'start': 46,
  'end': 

In [ ]:
new_string = ''

for n in output:
  result = n['word'].replace('▁' , ' ') + n['entity'].replace('0', '')
  new_string += result

new_string

" dear local newspaper, i think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people,, helps us learn about the globe(astronomy) and keeps us out of troble!. thing about! dont you think so?? how would you feel if your teenager is always on the phone with friends!? do you ever time to chat with your friends or buisness partner about things?. well now - there's a new way to chat: the computer, their.s. plenty of sites on the internet to do so: facebook, my,space, ect.. just think now, while your setting up meeting with your boss on the computer,, your teenager is having fun on the phone, not rushing to get off cause you want to use it. how did you learn about other countrys/states outside of yours?? well, i have by computer/internet, it's a new way to learn about what going on in our time!. you might think your child spends a lot of time on the computer, but ask them so. question about the economy, sea floor spreadin

#### Checking Capitalization Mistakes